```
conda install -c gurobi gurobi
```

In [2]:
import gurobipy as gp
import numpy as np

In this tutorial, I'll use the Gurobi solver. Students and academics can register for a free license [here](https://www.gurobi.com/academia/academic-program-and-licenses/). 

Pyomo also suports several other solvers. The solvers available for your version of pyomo can be listed by running the following command in an anaconda prompt using an environment with pyomo installed.
```
pyomo help --solvers
```

# Minimal Example

$Maximize\left( 40 \ x \right)$

s.t. 

$x \leq 40$

In [16]:
model = gp.Model()

ANSWER = 40

x = model.addVar(vtype=gp.GRB.CONTINUOUS,name="x")

model.setObjective(
    40 * x,
    gp.GRB.MAXIMIZE
)

model.addConstr( x <= 40)

x.setAttr("PStart",ANSWER)

model.setParam(gp.GRB.Param.LPWarmStart, 1)

model.optimize()
print("\n\nx =",x.x)

Warning on update mode = 1 after new variables or constraints added:
Setting LP warm start basis or start ignored
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 1 columns and 1 nonzeros
Model fingerprint: 0x3c3f69f8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 4e+01]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.600000000e+03


x = 40.0


# Array of Variables Example

$Maximize\left(\sum_{i \in I} c_i x_i \right)$

s.t. 

$x_i \leq b_i \ \ \ \ \forall \ i \in I$

In [20]:
model = gp.Model()

numVar = 10

I = range(numVar)

x = model.addVars(I, vtype=gp.GRB.CONTINUOUS, name="x")

c = np.random.uniform(0,1,numVar)
b = np.random.uniform(0,1,numVar)

model.setObjective(
    sum([c[i] * x[i] for i in I]),
    gp.GRB.MAXIMIZE
)

def myConstraint(i):
    return x[i] <= b[i]

for i in I:
    model.addConstr(myConstraint(i))

model.optimize()
for i in range(len(x)):
    xi = x[i]
    print("x{} =".format(i),xi.x)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 10 rows, 10 columns and 10 nonzeros
Model fingerprint: 0x2e5aff4d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 1e+00]
Presolve removed 10 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1416277e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.141627714e+00
x0 = 0.5806706539418138
x1 = 0.8607280208031708
x2 = 0.24201592458664556
x3 = 0.29066486985823825
x4 = 0.9960511308033559
x5 = 0.8213110857074437
x6 = 0.9240006723160997
x7 = 0.6273649998825911
x8 = 0.90508440928

# Multi-Dimensional Array of Variables Example

$Maximize\left(\sum_{i \in I} \sum_{j \in J} \sum_{k \in K} c_{i,j,k} x_{i,j,k} \right)$

s.t. 

$\sum_{k \in K} x_{i,j,k} \leq b_{i,j} \ \ \ \ \forall \ i \in \ I, \ j \ \in \ J$

In [44]:
model = gp.Model()

numI = 3
numJ = 4
numK = 5

I = range(numI)
J = range(numJ)
K = range(numK)

I_J_K = np.zeros((numI*numJ*numK,3))
ii = 0
for i in I:
    for j in J:
        for k in K:
            I_J_K[ii] = [i,j,k]
            ii += 1
        
x = model.addVars(numI,numJ,numK, vtype=gp.GRB.CONTINUOUS, name="x")

c = np.random.uniform(0,1,(numI,numJ,numK))
b = np.random.uniform(0,1,(numI,numJ))

model.setObjective(
    sum([c[i,j,k] * x[i,j,k] for i in I for j in J for k in K]),
    gp.GRB.MAXIMIZE
)

def myConstraint(i,j):
    return sum([x[i,j,k] for k in K]) <= b[i,j]
for i in I:
    for j in J:
        model.addConstr(myConstraint(i,j))

model.optimize()
for i in I:
    for j in J:
        for k in K:
            xijk = x[i,j,k]
            print("x{}-{}-{} =".format(i,j,k),xijk.x)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 12 rows, 60 columns and 60 nonzeros
Model fingerprint: 0xf1de87f5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-02, 8e-01]
Presolve removed 12 rows and 60 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.4427485e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.442748501e+00
x0-0-0 = 0.0
x0-0-1 = 0.8124810237167353
x0-0-2 = 0.0
x0-0-3 = 0.0
x0-0-4 = 0.0
x0-1-0 = 0.17879498884238665
x0-1-1 = 0.0
x0-1-2 = 0.0
x0-1-3 = 0.0
x0-1-4 = 0.0
x0-2-0 = 0.5383066971734936
x0-2-1 = 0.0
x0-2-2 = 0